In [ ]:
import sys
sys.path.append("../../")
from helper.utils import * 
from os import listdir 
from os.path import isfile, join, splitext
from sklearn.model_selection import StratifiedKFold
import pickle
from tqdm import tqdm

In [ ]:
data_dir="../../data/raw_data/"
test_pickle_path="../../data/raw_data/test_pickles/data_test_file.pkl"

In [ ]:
train_pickle_paths = join(data_dir, "train_pickles")
train_pickle_paths = [
    join(train_pickle_paths,f)
    for f in listdir(train_pickle_paths)
    if isfile(join(train_pickle_paths, f)) and splitext(f)[1] == ".pkl"
]

In [ ]:
def _read_pickle_(f):
    pkl_path= open(f, 'rb')
    return pickle.load(pkl_path)

train_df=[]

In [ ]:
images=[]
event_ids=[]
image_ids=[]
targets=[]
for path  in tqdm(train_pickle_paths) :
    data,target=_read_pickle_(path) 
    event_id=int(path.split("/")[-1].split(".")[0].replace("event",""))
    image_id=lambda x : str(event_id)+"_"+str(x)
    for i,image in enumerate(data) :
        event_ids.append(event_id)
        image_ids.append(image_id(i))
        images.append(image)
    targets.extend(target)   
        
images_arra=np.array(images).reshape((-1,100))        

In [ ]:
train_data=pd.DataFrame(data=images_arra,columns=["feat_"+str(i) for i in range(100)])
train_data["target"]=targets
train_data["image_id"]=image_ids
train_data["event_id"]=event_ids

In [ ]:
dic_types={11: "electron", 13 : "muon", 211:"pion", 321:"kaon",2212 : "proton"}
dic_types={11: 0, 
           13 : 1, 
           211:2, 
           321:3,
           2212 : 4}

train_data.target=train_data.target.map(dic_types)

In [ ]:
np.random.seed(1994)

In [ ]:
sample_number=3138
balanced_train=train_data[train_data.target.isin([0,1])]
pion_train=train_data[train_data.target==2]
kaon_train=train_data[train_data.target==3]
proton_train=train_data[train_data.target==4]
pion_train=pion_train.sample(frac=sample_number/len(pion_train))
kaon_train=kaon_train.sample(frac=sample_number/len(kaon_train))
proton_train=proton_train.sample(frac=sample_number/len(proton_train))
balanced_train=pd.concat([balanced_train,pion_train,kaon_train,proton_train])
balanced_train=balanced_train.sample(frac=1).reset_index(drop=True)
balanced_train.to_pickle("../proc_data/balanced_train.pkl")
len(balanced_train)

In [ ]:
event_ids=[]
images=[]
image_ids=[]

events=_read_pickle_(test_pickle_path) 
event_id="test_event"
for event in (events) : 
    image_id,image=event
    event_ids.append(event_id)
    images.append(image)
    image_ids.append(image_id)
images_arra=np.array(images).reshape((-1,100))        

In [ ]:
test_data=pd.DataFrame(data=images_arra,columns=["feat_"+str(i) for i in range(100)])
test_data["image_id"]=image_ids
test_data["event_id"]=event_ids
test_data.to_pickle("../proc_data/test.pkl")